<a href="https://colab.research.google.com/github/Armandohaj/Proyecto-Talleretapa/blob/main/Ultimaedicion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: When **using** this notebook, you must select a GPU Hardware Accelerator instead of CPU. Go to Runtime->Change Runtime Type and select any option that uses GPU.

# Dependencies

In [ ]:
!pip install face_recognition # Install face_recognition package using pip
import face_recognition
import cv2 # Import OpenCV
import numpy as np
from matplotlib import pyplot as plt # pyplot
from pickle import load, dump

# ipywidgets packages
import ipywidgets as widgets
from IPython.display import display, clear_output

# Live Streaming dependencies
from IPython.display import Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io
import html
import time


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566171 sha256=2fe8bc5de65d455939863b4c110b93cc35ec62297029872b4c098952aecbf6f8
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Define la variable I (asegúrate de darle un valor apropiado)
i = 1
# Ruta de la imagen
img_path = '/content/drive/MyDrive/Colab Notebooks/Images/' + 'img' + str(i) + '.jpg'

# Aquí debes tener una imagen válida en la variable 'img' antes de guardarla
# cv2.imwrite(img_path, img)

def show_image(image_path):
    """ Esta función muestra una imagen.
      Argumentos:
          image_path {str} -- Path de la imagen.
    """
    img = cv2.imread(image_path)
    img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img_cvt)
    plt.show()

# Define las coordenadas x1, y1, x2, y2
x1 = 100
y1 = 100
x2 = 200
y2 = 200

# Lista para guardar las rutas de las imágenes
img_list = []

# Agrega la ruta de la imagen a la lista
img_list.append(img_path)

def show_image_with_square(image_path, faces):
    """ Esta función muestra una imagen con un cuadro en cada rostro.
      Argumentos:
          image_path {str} -- Path de la imagen.
          faces {list} -- Lista de diccionarios con las coordenadas de los rostros.
    """
    img = cv2.imread(image_path)
    im2Display = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imTemp = im2Display.copy()
    for face in faces:
        x1 = face['x1']
        y1 = face['y1']
        x2 = face['x2']
        y2 = face['y2']
        imTemp = cv2.rectangle(imTemp, (x1, y1), (x2, y2), (0, 255, 0), 2)
    plt.imshow(imTemp)
    plt.show()

# Llama a las funciones según sea necesario
# show_image(img_path)
# face_rectangle = {'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2}
# show_image_with_square(img_path, [face_rectangle])



# Utils

## In-memory data

In [ ]:
registered_faces = {}
registered_professors = {}
registered_students = {}
registered_miscellaneous  = []
registered_name_professor = []
lista_estudiantes = []
lista_profesores = []
numero_imagen = 1


## Real time streaming

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 800, 600);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 800; //video.videoWidth;
      captureCanvas.height = 600; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

In [ ]:
# JavaScript code to stop the webcam stream
js_stop_stream = """
    const stream = video.srcObject;
    const tracks = stream.getTracks();

    tracks.forEach(function(track) {
        track.stop();
    });

    video.srcObject = null;
    div.style.display = 'none';  // Hide the container
"""

# Function to stop the video streaming
def stop_video_stream():
    display(Javascript(js_stop_stream))

# Register Faces

In [ ]:
from ipywidgets.widgets import widget
import pickle

uploaded_image_data = ""
# Create a text input widget for the name
name_input = widgets.Text(
    placeholder='Ingrese su nombre',
    description='Nombre:',
    disabled=False
)

# Create a file upload widget
upload_button = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Cargar Imagen'
)

or_text = widgets.HTML(
    value=" o "
)

# Create a button to trigger face registration
register_button = widgets.Button(description='Registrar Cara')

# Create a button to trigger image capture
capture_button = widgets.Button(description='Capturar Imagen')

# Create an output widget to display the uploaded or captured face image
output = widgets.Output()

# Initialize variables to store the uploaded image and name
uploaded_image_data = None
registered_name_professor = None
registered_name_student = None
registered_miscellaneous = None


# Import necessary libraries (ensure you have them installed)
import numpy as np
import cv2
import face_recognition

# Define a function to handle image upload
def handle_upload(change):

    global numero_imagen
    global uploaded_image_data
    uploaded_filename = list(upload_button.value.keys())[0]
    uploaded_image_data = upload_button.value[uploaded_filename]['content']

    with output:
        clear_output()
        display(widgets.Image(value=uploaded_image_data))

    # Assuming uploaded_image_data is binary image data in bytes
    image_array = np.frombuffer(uploaded_image_data, np.uint8)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    # Convert the image to RGB format
    uploaded_image_data = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #print(numero_imagen)
    cv2.imwrite('/content/drive/MyDrive/imagenes/'+ 'numero'+str(numero_imagen)+'.jpg',image )
    numero_imagen += 1

# Define a function to handle image capture
def handle_capture(button):
    global numero_imagen
    global uploaded_image_data
    # Start streaming video from the webcam
    video_stream()
    js_reply = video_frame("Capturando...", "")

    stop_video_stream()  # Stop the video stream after capturing the frame

    if not js_reply:
        with output:
            clear_output()
            print("Error al capturar la imagen!")

    # Convert JS response to OpenCV Image

    uploaded_image_data = js_to_image(js_reply["img"])
    #trabar esto para los archivos txt
    cv2.imwrite('/content/drive/MyDrive/imagenes/'+ 'numero'+str(numero_imagen)+'.jpg',uploaded_image_data )
    numero_imagen += 1

    with output:
        clear_output()
        # Encode the captured frame as PNG
        _, buffer = cv2.imencode('.png', uploaded_image_data)

        # Convert the encoded image to bytes
        image_bytes = buffer.tobytes()
        #display(widgets.Image(value=image_bytes))

# Define a function to check if a face is already registered
def check_if_face_registered(encoding):
    global registered_faces
    for face_name, face_encoding in registered_faces.items():
        print(encoding,"-",face_encoding.get("encoding"))
        matches = face_recognition.compare_faces([face_encoding.get("encoding")], encoding)
        if any(matches):
            return True
    return False

# Define a function to handle face registration
def handle_register(button):

    global registered_faces
    global registered_name_professor
    global registered_name_student
    global registered_name_miscellaneous

    if uploaded_image_data is not None:
        registered_name = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
          with output:
              clear_output()
              print("Esta cara ya está registrada.")
              return

        with output:
            clear_output()
            print(f"Cara registrada para {registered_name}.")

            # Here you can add logic to associate the role with the registered face
            # For example, you can use the role_dropdown value
            # Example: role = role_dropdown.value

            # Store the face encoding along with the role
            registered_faces[registered_name] = {
                'encoding': face_encodings[0],
                'role': role_dropdown.value  # Use the selected role
            }

# Create a dropdown widget for role selection
role_dropdown = widgets.Dropdown(
    options=['Estudiante', 'Profesor', 'Misceláneo'],
    description='Rol:',
    disabled=False
)

# Set up the widget event handlers
upload_button.observe(handle_upload, names='value')
capture_button.on_click(handle_capture)
register_button.on_click(handle_register)

# Create a button to trigger face registration for professors
register_button_professor = widgets.Button(description='Registrar Profesor')

# Define a function to handle face registration for professors
def handle_register_professor(button):

    global registered_faces
    global registered_name_professor

    if uploaded_image_data is not None:
        # Obtener el nombre del profesor desde el campo de entrada
        registered_name_professor = name_input.value

        # Detectar caras en la imagen cargada
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # Si no se detecta ninguna cara, mostrar una ventana emergente
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # Si se detecta más de una cara, mostrar una ventana emergente
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Obtener las codificaciones de las caras
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Comprobar si la cara ya está registrada
        if check_if_face_registered(face_encodings[0]):
          with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return


        print(f"Cara registrada para {registered_name_professor} (Profesor).")

        # Recopilar información adicional sobre el profesor

        identificacion = widgets.Text(description = "Identificacion: ")
        fecha_de_nacimiento = widgets.Text(description = "Nacimiento: ")
        nacionalidad = widgets.Text(description = "Nacionalidad: ")
        telefono = widgets.Text(description = "Teléfono: ")
        celular = widgets.Text(description = "Celular: ")
        email = widgets.Textarea(description = "Email: ")
        direccion = widgets.Text(description = "Dirección: ")
        especialidad = widgets.Text(description = "Especialidad: ")
        años_dando_clases = widgets.Text(description = "Años dando clases: " )
        genero = widgets.Text(description = "Genero: ")
        lista_profesores = []
        def guardar_handler(_):
          #Almacenar la codificación de la cara junto con la información del profesor
          profesor ={
              'nombre': registered_name_professor,
              'encoding': face_encodings[0],
              'role': 'Profesor',
              'informacion_adicional': {
                  'identificacion': identificacion.value,
                  'fecha_de_nacimiento': fecha_de_nacimiento.value,
                  'nacionalidad': nacionalidad.value,
                  'telefono': telefono.value,
                  'celular': celular.value,
                  'email': str(email.value).split(),
                  'direccion': direccion.value,
                  'especialidad': especialidad.value,
                  'años_dando_clases': años_dando_clases.value,
                  'genero': genero.value,
              }
          }

          with open("/content/drive/MyDrive/Informacion/archivoprofesor.txt", "wb") as f:
              pickle.dump(profesor, f)
          lista_profesores.append(profesor)

          registered_faces[registered_name_professor] = profesor
          registered_professors[registered_name_professor] = profesor


          print("La informacion se guardo correctamente")


        guardar_professor_button = widgets.Button(description='Guardar informacion')
        guardar_professor_button.on_click(guardar_handler)
        display(registered_name_professor, identificacion, fecha_de_nacimiento, nacionalidad, telefono, celular, email, direccion, especialidad, años_dando_clases, genero, guardar_professor_button )


register_button_students = widgets.Button(description='Registrar Estudiante')

# Define a function to handle face registration for students
def handle_register_student(button):
    global registered_faces
    global registered_name_student

    if uploaded_image_data is not None:
        registered_name_student = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return
        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)
        print(face_encodings)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return



        print(f"Cara registrada para {registered_name_student} (estudiante).")


        #Recopilar información adicional sobre el estudiante
        identificacion = widgets.Text(description = "Identificacion: ")
        nombre = widgets.Text(description = "Nombre: ")
        fecha_de_nacimiento = widgets.Text(description = "Nacimiento: ")
        nacionalidad = widgets.Text(description = "Nacionalidad: ")
        telefono = widgets.Text(description = "Teléfono: ")
        celular = widgets.Text(description = "Celular: ")
        email = widgets.Textarea(description = "Email: ")
        direccion = widgets.Text(description = "Dirección: ")
        carrera = widgets.Text(description = "Carrera: ")
        pasatiempos = widgets.Text (description = "Pasatiempos: ")
        signo_zodiacal = widgets.Text(description = "Signo zodiacal: ")
        lista_estudiantes = []
        def guardar_student(_):

          #Almacenar la codificación de la cara junto con la información del estudiante
          student = {
              'nombre': registered_name_student,
              'encoding': face_encodings[0],
              'role': 'Student',
              'informacion_adicional': {
                  'identificacion': identificacion.value,
                  'nombre': nombre.value,
                  'fecha_de_nacimiento': fecha_de_nacimiento.value,
                  'nacionalidad': nacionalidad.value,
                  'telefono': telefono.value,
                  'celular': celular.value,
                  'email': str(email.value).split(),
                  'direccion': direccion.value,
                  'carrera': carrera.value,
                  'pasatiempos': pasatiempos.value,
                  'signo_zodiacal': signo_zodiacal.value

              }
          }
          #with open("/content/drive/MyDrive/Informacion/archivoestudiante.txt", "wb") as f:
           #   pickle.dump(student, f)
          lista_estudiantes.append(student)



          registered_faces[registered_name_student] = student
          registered_students[registered_name_student] = student


          print("La informacion se guardo correctamente")





        guardar_student_button = widgets.Button(description='Guardar informacion')
        guardar_student_button.on_click(guardar_student)
        display(identificacion, nombre, fecha_de_nacimiento, nacionalidad, telefono, celular, email, direccion, carrera, pasatiempos, signo_zodiacal, guardar_student_button )

register_button_miscellaneo = widgets.Button(description='Registrar Micelanio')
# Define a function to handle face registration for micelaneo
def handle_register_miscellaneous(button):
    global registered_faces
    global registered_name_miscellaneous

    if uploaded_image_data is not None:
        registered_name_miscellaneous = name_input.value

        # Detect faces in the frame
        face_locations = face_recognition.face_locations(uploaded_image_data)

        # If there is no face, raise a popup window
        if len(face_locations) == 0:
            with output:
                clear_output()
                print("No se detectó ninguna cara en la imagen.")
                return

        # If there is more than one face, raise a popup window
        if len(face_locations) > 1:
            with output:
                clear_output()
                print("Se detectaron más de una cara en la imagen. Registre solo una cara a la vez.")
                return

        # Get the face encodings
        face_encodings = face_recognition.face_encodings(uploaded_image_data)

        # Check if the face is registered
        if check_if_face_registered(face_encodings[0]):
            with output:
                clear_output()
                print("Esta cara ya está registrada.")
                return

        with output:
            clear_output()
            print(f"Cara registrada para {registered_name_miscellaneous} (miscellaneous).")

            # Store the face encoding along with the role
            registered_faces[registered_name_miscellaneous] = {
                'encoding': face_encodings[0],
                'role': 'Micelanio'
            }

        # Recopilar información adicional sobre el micelaneo

        identificacion = widgets.Text(description = "Identificacion: ")
        nombre = widgets.Text(description = "Nombre: ")
        fecha_de_nacimiento = widgets.Text(description = "Nacimiento: ")
        nacionalidad = widgets.Text(description = "Nacionalidad: ")
        telefono = widgets.Text(description = "Teléfono: ")
        celular = widgets.Text(description = "Celular: ")
        email = widgets.Textarea(description = "Email: ")
        direccion = widgets.Text(description = "Dirección: ")
        turno = widgets.Text(description = "Turno: ")
        habilidad = widgets.Text(description = "Habilidad: ")
        estado_civil = widgets.Text(description = "estado civil: ")

        def guardar_miscellaneous(_):

          #Almacenar la codificación de la cara junto con la información del micelaneo
          miscelaneo = {
              'nombre': registered_name_miscellaneous,
              #'encoding': face_encodings[0],
              'role': 'Micelaneo',
              'informacion_adicional': {
                  'identificacion': identificacion.value,
                  'nombre': nombre.value,
                  'fecha_de_nacimiento': fecha_de_nacimiento.value,
                  'nacionalidad': nacionalidad.value,
                  'telefono': telefono.value,
                  'celular': celular.value,
                  'email': str(email.value).split(),
                  'direccion': direccion.value,
                  'turno': turno.value,
                  'habilidad': habilidad.value,
                  'estado_civil': estado_civil.value
              }
          }
          #with open("/content/drive/MyDrive/Informacion/archivomiscelaneo.txt", "wb") as f:
              #pickle.dump(miscelaneo, f)
          #arch_binario("/content/drive/MyDrive/Informacion/archivomiscelaneo.txt")
          registered_faces[registered_name_miscellaneous] = miscelaneo
          registered_students[registered_name_miscellaneous] = miscelaneo


          print("La informacion se guardo correctamente")

        guardar_micelaneo_button = widgets.Button(description='Guardar informacion')
        guardar_micelaneo_button.on_click(guardar_miscellaneous)
        display(identificacion, fecha_de_nacimiento, nacionalidad, telefono, celular, email, direccion, turno, habilidad, estado_civil, guardar_micelaneo_button )


# Set up the widget event handler for registering professors
register_button_professor.on_click(handle_register_professor)
register_button_students.on_click(handle_register_student)
register_button_miscellaneo.on_click(handle_register_miscellaneous)
# Create a container for the widgets
container = widgets.VBox([name_input, role_dropdown, widgets.HBox([upload_button, or_text, capture_button]),
                          widgets.HBox([register_button, or_text, register_button_professor, or_text, register_button_students, or_text, register_button_miscellaneo]), output])

# Display the container
display(container)


[-1.05218522e-01  1.05360791e-01  1.84627175e-02 -5.27196378e-02
 -1.60235912e-01  1.10741206e-01 -5.22380546e-02 -4.60209697e-02
  1.32975519e-01 -6.24045394e-02  2.44922966e-01 -4.41574231e-02
 -2.41434991e-01 -1.30311772e-02 -5.59172966e-02  1.13486178e-01
 -1.33928984e-01 -1.85182542e-02 -1.52121440e-01 -1.51756674e-01
 -1.01048775e-01  9.54107493e-02  1.90002434e-02  3.54325920e-02
 -1.51693106e-01 -2.93140858e-01 -3.55761908e-02 -1.53284222e-01
  1.27999574e-01 -1.60937071e-01 -1.73180550e-02  4.55032103e-02
 -1.11721382e-01 -1.54408887e-02  5.40027283e-02  1.58250928e-02
 -3.55267972e-02 -1.20738640e-01  2.09154487e-01 -6.92404658e-02
 -7.93076754e-02  6.55375123e-02  1.34319574e-01  2.34614134e-01
  1.07906938e-01 -5.07750362e-03  2.45023519e-04 -9.69231427e-02
  2.13245377e-01 -1.96304739e-01  1.06964685e-01  8.73287693e-02
  1.42760992e-01  9.78428349e-02  7.66315535e-02 -1.67435691e-01
  1.37400925e-02  1.64987147e-01 -2.38368303e-01  8.12831074e-02
  3.18146124e-02  2.54925

'cr7'

Text(value='', description='Identificacion: ')

Text(value='', description='Nacimiento: ')

Text(value='', description='Nacionalidad: ')

...

Text(value='', description='Especialidad: ')

Text(value='', description='Años dando clases: ')

Text(value='', description='Genero: ')

Button(description='Guardar informacion', style=ButtonStyle())

La informacion se guardo correctamente


In [ ]:
def arch_binario(archivo_encriptado):
  with open(archivo_encriptado,"rb") as archivo_ens:
    datos_encriptados = archivo_ens.read()
    print(datos_encriptados)



In [ ]:
ruta = "/content/drive/MyDrive/Informacioncurso/infocurso.txt"
arch_binario(ruta)

b''


In [ ]:

print(registered_professors)
print(registered_faces)

{'aro': {'nombre': 'aro', 'role': 'Profesor', 'informacion_adicional': {'identificacion': 'a', 'fecha_de_nacimiento': 'a', 'nacionalidad': 'a', 'telefono': 'a', 'celular': 'a', 'email': ['aa'], 'direccion': 'aa', 'especialidad': 'a', 'años_dando_clases': 'a', 'genero': 'a'}}}
{'Kenneth': {'nombre': 'Kenneth', 'encoding': array([-0.10316055,  0.08228562, -0.016289  , -0.07027595, -0.04784743,
       -0.07272617, -0.04694108, -0.07017116,  0.21087202, -0.10079197,
        0.25848392, -0.07194638, -0.25418597, -0.08368912, -0.06297559,
        0.09223561, -0.07012621, -0.08198915, -0.02695721, -0.07745903,
        0.11619341,  0.01036863, -0.00766334,  0.09851883, -0.10878635,
       -0.22769549, -0.09380886, -0.13481969,  0.02694675, -0.10621172,
       -0.0270943 ,  0.0260984 , -0.12476645, -0.11406602,  0.05521878,
        0.0815637 , -0.06512554, -0.00316601,  0.21681222,  0.00647226,
       -0.12466207, -0.03590716,  0.047163  ,  0.22701007,  0.15555316,
        0.06105843, -0.029736

### informacion del aula

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Lista para almacenar diccionarios de aula
aula_informacion = []

# Crear widgets
numero_de_aula = widgets.Text(description="Número: ")
carnet_miscelaneo = widgets.Text(description="Carnet: ")
capacidad = widgets.Text(description = "Capacidad: ")
aire = widgets.Text(description = "Aire: ")

guardar_button = widgets.Button(description="Guardar Información")
print("informacion del aula: ")

# Función para guardar la información
def guardar_aula(_):
    aula_info = {
        'numero': numero_de_aula.value,
        'carnet': carnet_miscelaneo.value,
        'capacidad': capacidad.value,
        'aire': aire.value
    }
    aula_informacion.append(aula_info)
    with open("/content/drive/MyDrive/Informacioncurso/archivoaula.txt", "wb") as f:
              pickle.dump(aula_informacion, f)
    print("Información del aula guardada con éxito.")

# Boton para guardar
guardar_button.on_click(guardar_aula)

# Posicion de los widgets
form = widgets.VBox([numero_de_aula, carnet_miscelaneo, capacidad, aire, guardar_button])

# imprimir
display(form)


informacion del aula: 


Información del aula guardada con éxito.


In [ ]:
print(aula_informacion)

[]


### informacion del curso

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# diccionario para guardar la informacion del curso
informacion_curso = []
print("informacion del curso: ")

#widgets
codigo_de_curso = widgets.Text(description = "Codigo: ")
nombre_de_curso = widgets.Text(description = "Nombre: ")
periodo = widgets.Text(description = "Periodo: ")
carrera_al_que_pertenece = widgets.Text(description = "Carrera: ")
horario = widgets.Text(description = "Horario: ")
aula = widgets.Text(description = "aula: ")
guardar_button = widgets.Button(description = "Guardar informacion")

# funcion para almacenar la informacion del curso
def guardar_curso(_):
    info_curso = {
      'codigo': codigo_de_curso.value,
      'nombre': nombre_de_curso.value,
      'periodo': periodo.value,
      'carrera': carrera_al_que_pertenece.value,
      'lista_estudiantes' : lista_estudiantes,
      'profesor_lista': lista_profesores,
      'horario': horario.value,
      'aula': aula.value,
    }
    informacion_curso.append(info_curso)


    print("informacion guardada con exito")


def obtener_estudiantes(curso, estudiantes):
    for estudiante, informacion in estudiantes.items():
        if curso.get("identificacion") == informacion.get("identificacion"):
            return informacion
    return None


def obtener_profesor(curso, profesores):
    for profesor, informacion in profesores.items():
        if curso.get("identificacion") == informacion.get("identificacion"):
            return informacion
    return None



# Boton para guardar
guardar_button.on_click(guardar_curso)

# Posicion de los widgets
form = widgets.VBox([codigo_de_curso, nombre_de_curso, periodo, carrera_al_que_pertenece, horario, aula, guardar_button])

# imprimir
display(form)




informacion del curso: 


In [ ]:
icape = informacion_curso[0]
print(obtener_profesor(icape, registered_professors))
print(obtener_estudiantes(icape, registered_students))

In [ ]:
print(informacion_curso[0])

In [ ]:

for face_name, face_encoding in registered_faces.items():
  print(face_name,"a", face_encoding, face_encoding.get("encoding"),"b")
  print(face_recognition.face_encodings(uploaded_image_data))


kenneth a {'nombre': 'kenneth', 'encoding': array([-0.06539143,  0.03959969,  0.0032866 , -0.12482777, -0.0510319 ,
       -0.02055511,  0.00291547, -0.08964118,  0.20280828, -0.09826846,
        0.22762406, -0.04988693, -0.23736438, -0.10438316, -0.02743619,
        0.0698716 , -0.03983317, -0.14495158, -0.02383701, -0.033531  ,
        0.05642552, -0.02979366, -0.058378  ,  0.12214065, -0.09563441,
       -0.31076509, -0.110818  , -0.1304123 ,  0.00353698, -0.05975014,
       -0.00297278,  0.06698079, -0.15480424, -0.06269004,  0.05484359,
        0.04792088, -0.00347601,  0.02977257,  0.20139134,  0.04715447,
       -0.1873941 , -0.02069403,  0.05432498,  0.21536022,  0.11186771,
        0.11500287, -0.00564508, -0.10863267,  0.15552163, -0.19410145,
        0.0883612 ,  0.19212249,  0.16092023,  0.07784297,  0.10918169,
       -0.14925753,  0.04559276,  0.08437502, -0.27999049,  0.09379458,
       -0.03238672, -0.05446094, -0.05081884, -0.03978182,  0.19069433,
        0.12165269, 

### Consultas

# Recognize Faces

In [ ]:
registered_faces = {
    "Nombre1": face_recognition.load_image_file("/content/drive/MyDrive/Imagenes/nombre1.jpg"),
    #"Nombre2": face_recognition.load_image_file("ruta_a_la_imagen2.jpg"),

}

###Reportar asistencia a curso


In [ ]:
import face_recognition
import cv2
import datetime

# Inicializa la lista para almacenar los registros de asistencia.
asistencia = []

# Configura la cámara (puedes usar 0 para la cámara predeterminada).
video_capture = cv2.VideoCapture(0)

# Carga las caras previamente almacenadas con sus nombres y rutas de imagen.
upload_imagine_data = [
    {"nombre": "Persona1", "ruta": "/content/drive/MyDrive/Imagenes/nombre1.jpg"},
    # Agrega más personas si es necesario.
]

known_face_names = []
known_face_encodings = []

for imagen_data in upload_imagine_data:
    nombre = imagen_data['nombre']
    ruta_imagen = imagen_data['ruta']

    imagen = face_recognition.load_image_file(ruta_imagen)
    encoding = face_recognition.face_encodings(imagen)[0]

    known_face_names.append(nombre)
    known_face_encodings.append(encoding)

while True:
    ret, frame = video_capture.read()
    rgb_frame = frame[:, :, ::-1]

    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

        name = "Desconocido"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_face_names[first_match_index]

        now = datetime.datetime.now()
        registro = {
            'nombre': name,
            'ruta_imagen': 'imagen_guardada.jpg',  # Cambia la ruta según tu preferencia.
            'fecha_hora': now.strftime("%Y-%m-%d %H:%M:%S")
        }
        asistencia.append(registro)

        # Dibuja un cuadro alrededor de la cara identificada y muestra el nombre.
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 0.5, (255, 255, 255), 1)

    cv2.imshow('Video de Asistencia', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

# La lista 'asistencia' ahora contiene los registros de asistencia.


In [ ]:
print(registered_faces)

###Consultas


In [ ]:
def mostrar_menu():
    print("Menú:")
    print("1. Opción 1")
    print("2. Opción 2")
    print("3. Opción 3")
    print("4. Salir")

def opcion_1():
  print("Has seleccionado la Opción 1.")
  for id_profesor, info_profesor in registered_professors.items():
        print(f"ID de Profesor: {id_profesor}")
        print(f"Nombre: {info_profesor['nombre']}")
        print(f"Apellido: {info_profesor['apellido']}")
        print(f"Departamento: {info_profesor['departamento']}")


def opcion_2():
  print("Has seleccionado la Opción 2.")
  for id_estudiante, info_estudiante in registered_students.items():
        print(f"ID de Estudiante: {id_estudiante}")
        print(f"Nombre: {info_estudiante['nombre']}")
        print(f"Apellido: {info_estudiante['apellido']}")
        print(f"Edad: {info_estudiante['edad']}")

def opcion_3():
  print("Has seleccionado la Opción 3.")
  for elemento in registered_miscellaneous:
        print("Información Miscelánea:")
        for key, value in elemento.items():
            print(f"{key}: {value}")
        print()
def opcion_4():
  for aula in informacion_curso:
        print("Información del Aula:")
        for key, value in aula.items():
            print(f"{key}: {value}")
        print()
def opcion_5():
    for curso in registered_miscellaneous:
        print(f"Curso: {curso['nombre_curso']}")
        print(f"Descripción: {curso['descripcion']}")

        # Mostrar información de profesores relacionados con el curso
        print("Profesores:")
        for id_profesor in curso['profesores']:
            profesor = registered_professors.get(id_profesor)
            if profesor:
                print(f"- Nombre del Profesor: {profesor['nombre']} {profesor['apellido']}")

        # Mostrar información de estudiantes relacionados con el curso
        print("Estudiantes:")
        for id_estudiante in curso['estudiantes']:
            estudiante = registered_students.get(id_estudiante)
            if estudiante:
                print(f"- Nombre del Estudiante: {estudiante['nombre']} {estudiante['apellido']}")
        print("\n")

#def opcion_7():
# Definición de una función para ordenar estudiantes por edad utilizando el método de burbuja
def ordenar_estudiantes_por_edad(estudiantes):
    n = len(estudiantes)
    for i in range(n - 1):
        for j in range(0, n - i - 1):
            if estudiantes[j]['edad'] > estudiantes[j + 1]['edad']:
                estudiantes[j], estudiantes[j + 1] = estudiantes[j + 1], estudiantes[j]

# Suponiendo que registered_courses contiene la información de los cursos y cada curso tiene una lista de IDs de estudiantes
for curso in registered_courses:
    print(f"Curso: {curso['nombre_curso']}")
    print(f"Descripción: {curso['descripcion']}")
    print("Estudiantes:")

    # Obtener la información de los estudiantes asociados a este curso
    estudiantes_curso = []
    for id_estudiante in curso['estudiantes']:
        estudiante = registered_students.get(id_estudiante)
        if estudiante:
            estudiantes_curso.append(estudiante)

    # Ordenar los estudiantes por edad
    ordenar_estudiantes_por_edad(estudiantes_curso)

    # Mostrar la información de los estudiantes ordenados por edad
    for estudiante in estudiantes_curso:
        print(f"- Nombre del Estudiante: {estudiante['nombre']} {estudiante['apellido']}")
        print(f"  Edad: {estudiante['edad']}")

    print("\n")
def opcion_8():
# Supongamos que tienes una lista de estudiantes con su información y asistencias a cursos.
# registered_students es un diccionario donde la clave es el ID del estudiante y el valor es un diccionario con su información.
# registered_courses_attendance es un diccionario donde la clave es el ID del estudiante y el valor es una lista de cursos a los que asistió el estudiante.
    estudiante_max_asistencias = None
    max_asistencias = -1  # Inicializamos con un valor negativo para asegurarnos de encontrar un estudiante con asistencias.

    for id_estudiante, cursos_asistidos in registered_courses_attendance.items():
        if len(cursos_asistidos) > max_asistencias:
            max_asistencias = len(cursos_asistidos)
            estudiante_max_asistencias = id_estudiante

    if estudiante_max_asistencias is not None:
        # Mostrar la información del estudiante con más asistencias.
        estudiante_info = registered_students.get(estudiante_max_asistencias)
        print("Información del Estudiante con Más Asistencias:")
        print(f"ID de Estudiante: {estudiante_max_asistencias}")
        for key, value in estudiante_info.items():
            print(f"{key}: {value}")
        print(f"Número de Asistencias: {max_asistencias}")
    else:
        print("No se encontraron estudiantes con asistencias.")
def opcion_9():
  while True:
    mostrar_menu()
    opcion = input("Selecciona una opción: ")
    if opcion == "1":
        opcion_1()
    elif opcion == "2":
        opcion_2()
    elif opcion == "3":
        opcion_3()
    elif opcion == "4":
        print("Saliendo del programa.")
        break
    else:
        print("Opción no válida. Por favor, selecciona una opción válida.")
